In [1]:
%reload_ext autoreload
%autoreload 2
from MultimodalDataset.FinanceData import FinanceDataProcessor
from MultimodalDataset.MedicalData import MedicalDataProcessor
from MultimodalDataset.ClimateData import ClimateDataProcessor
from src.utils import load_config, get_max_token_size
import yaml
from datasets import Dataset, DatasetDict, load_dataset
import pandas as pd

# For Climate Dataset

In [6]:
file_path = "config/default/climate.yaml"
# Load the existing YAML file
with open(file_path, 'r') as file:
    config = yaml.safe_load(file)

# Iterate over input_window and output_window from 1 to 7
for i in range(1, 8):
    # Update the values in the config
    config['dataset']['input_window'] = i
    config['dataset']['output_window'] = i
    config['dataset']['hf_repo'] = f"Howard881010/climate-{i}day-inContext"
    
    # Save the updated configuration back to the original file
    with open(file_path, 'w') as outfile:
        yaml.dump(config, outfile, default_flow_style=False)
    
    print(f"Saved configuration with input_window={i} and output_window={i}.")
    climate_cfg = load_config('config/default/climate.yaml')

    processor = ClimateDataProcessor(climate_cfg['dataset'])
    climate_dataset = processor.get_dataset()
    max_length_1 = get_max_token_size(climate_dataset, 'input_text_time', 'output_text_time', 'instruction-2')
    max_length_2 = get_max_token_size(climate_dataset, 'input_text_time', 'output_time', 'instruction-4')

    # processor.push_to_huggingface(climate_dataset)
    print(max_length_1, max_length_2)


Saved configuration with input_window=1 and output_window=1.
instruction_1:  Given the weather information of the first 1 day, predict the weather information of the next 1 day. Output the result strictly in the following JSON format and no additional text:
{
    "day_2_date": "YYYY-MM-DD",
    "day_2_weather_forecast": "Weather description"
}
instruction_2:  Given the weather information of the first 1 day, predict the weather information of the next 1 day. Output the result strictly in the following JSON format and no additional text:
{
    "day_2_date": "YYYY-MM-DD",
    "day_2_weather_forecast": "Weather description",
    "day_2_temp": "A Float Number"
}
instruction_3:  Given the weather information of the first 1 day, predict the weather information of the next 1 day. Output the result strictly in the following JSON format and no additional text:
{
    "day_2_date": "YYYY-MM-DD",
    "day_2_weather_forecast": "Weather description"
}
instruction_4:  Given the weather information of

KeyboardInterrupt: 

In [ ]:
# Load the dataset
for i in range(1, 8):
    data = load_dataset(f'Howard881010/climate-{i}day-finetuned')

    # Initialize dictionaries to hold processed data
    data_all = {'train': [], 'test': [], 'valid': []}

    # Process each split
    for split in ['train', 'test', 'valid']:
        data_split = pd.DataFrame(data[split])
        
        t2t = data_split[['input_text', 'output_text', 'instruction-1']].rename(columns={'instruction-1': 'instruction', 'input_text': 'input', 'output_text': 'output'})
        tT2tT = data_split[['input_text_time', 'output_text_time', 'instruction-2']].rename(columns={'instruction-2': 'instruction', 'input_text_time': 'input', 'output_text_time': 'output'})
        tT2t = data_split[['input_text_time', 'output_text', 'instruction-3']].rename(columns={'instruction-3': 'instruction', 'input_text_time': 'input', 'output_text': 'output'})
        tT2T = data_split[['input_text_time', 'output_time', 'instruction-4']].rename(columns={'instruction-4': 'instruction', 'input_text_time': 'input', 'output_time': 'output'})
        
        # Append the transformed data for each split
        data_all[split].append(t2t)
        data_all[split].append(tT2tT)
        data_all[split].append(tT2t)
        data_all[split].append(tT2T)
        
        # Concatenate all processed DataFrames for the current split
        data_all[split] = pd.concat(data_all[split])

    # Convert each processed DataFrame back to Hugging Face Dataset format
    dataset_dict = DatasetDict({
        'train': Dataset.from_pandas(data_all['train']),
        'test': Dataset.from_pandas(data_all['test']),
        'valid': Dataset.from_pandas(data_all['valid'])
    })

    # Push the processed dataset to Hugging Face Hub
    dataset_dict.push_to_hub(f'Howard881010/climate-{i}day')


# For Medical dataset

In [2]:
import yaml

file_path = "config/default/medical.yaml"
# Load the existing YAML file
with open(file_path, 'r') as file:
    config = yaml.safe_load(file)

# Iterate over input_window and output_window from 1 to 7
for i in range(1, 8):
    # Update the values in the config
    config['dataset']['input_window'] = i
    config['dataset']['output_window'] = i
    config['dataset']['hf_repo'] = f"Howard881010/medical-{i}day-inContext"
    
    # Save the updated configuration back to the original file
    with open(file_path, 'w') as outfile:
        yaml.dump(config, outfile, default_flow_style=False)
    
    print(f"Saved configuration with input_window={i} and output_window={i}.")
    medical_cfg = load_config('config/default/medical.yaml')

    processor = MedicalDataProcessor(medical_cfg['dataset'])
    medical_dataset = processor.get_dataset()
    max_length_1 = get_max_token_size(medical_dataset, 'input_text_time', 'output_text_time', 'instruction-2')
    max_length_2 = get_max_token_size(medical_dataset, 'input_text_time', 'output_time', 'instruction-4')
    processor.push_to_huggingface(medical_dataset)
    print(max_length_1, max_length_2)

Saved configuration with input_window=1 and output_window=1.
Given the medical information of the first 1 day, predict the medical information of the next 1 day. Output the result strictly in the following JSON format and no additional text:
{
    "day_2_date": "YYYY-MM-DD",
    "day_2_medical_notes": "Medical description"
}
Given the medical information of the first 1 day, predict the medical information of the next 1 day. Output the result strictly in the following JSON format and no additional text:
{
    "day_2_date": "YYYY-MM-DD",
    "day_2_medical_notes": "Medical description",
    "day_2_Heart_Rate": "A Float Number"
}
Given the medical information of the first 1 day, predict the medical information of the next 1 day. Output the result strictly in the following JSON format and no additional text:
{
    "day_2_date": "YYYY-MM-DD",
    "day_2_medical_notes": "Medical description"
}
Given the medical information of the first 1 day, predict the medical information of the next 1 day

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

6404 3623


In [9]:
# Load the dataset
for i in range(1, 8):
    data = load_dataset(f'Howard881010/medical-{i}day-finetuned')

    # Initialize dictionaries to hold processed data
    data_all = {'train': [], 'test': [], 'valid': []}

    # Process each split
    for split in ['train', 'test', 'valid']:
        data_split = pd.DataFrame(data[split])
        
        t2t = data_split[['input_text', 'output_text', 'instruction-1']].rename(columns={'instruction-1': 'instruction', 'input_text': 'input', 'output_text': 'output'})
        tT2tT = data_split[['input_text_time', 'output_text_time', 'instruction-2']].rename(columns={'instruction-2': 'instruction', 'input_text_time': 'input', 'output_text_time': 'output'})
        tT2t = data_split[['input_text_time', 'output_text', 'instruction-3']].rename(columns={'instruction-3': 'instruction', 'input_text_time': 'input', 'output_text': 'output'})
        tT2T = data_split[['input_text_time', 'output_time', 'instruction-4']].rename(columns={'instruction-4': 'instruction', 'input_text_time': 'input', 'output_time': 'output'})
        
        # Append the transformed data for each split
        data_all[split].append(t2t)
        data_all[split].append(tT2tT)
        data_all[split].append(tT2t)
        data_all[split].append(tT2T)
        
        # Concatenate all processed DataFrames for the current split
        data_all[split] = pd.concat(data_all[split])

    # Convert each processed DataFrame back to Hugging Face Dataset format
    dataset_dict = DatasetDict({
        'train': Dataset.from_pandas(data_all['train']),
        'test': Dataset.from_pandas(data_all['test']),
        'valid': Dataset.from_pandas(data_all['valid'])
    })

    # Push the processed dataset to Hugging Face Hub
    dataset_dict.push_to_hub(f'Howard881010/medical-{i}day')

Generating train split:   0%|          | 0/6042 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/751 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/796 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/713 [00:00<?, ?B/s]

# For Financial dataset

In [3]:
file_path = "config/default/finance.yaml"
# Load the existing YAML file
with open(file_path, 'r') as file:
    config = yaml.safe_load(file)

# Iterate over input_window and output_window from 1 to 7
for ticker in ["AAPL", "AMD", "AMZN", "AVGO", "NFLX"]:
    for i in range(1, 8):
        # Update the values in the config
        config['dataset']['input_window'] = i
        config['dataset']['output_window'] = i
        config['dataset']['hf_repo'] = f"Howard881010/finance-{ticker}-{i}day-inContext"
        config['dataset']['ticker'] = ticker
        config['dataset']['dataset_path'] = f"/home/ubuntu/multimodal/Data/financial-grouped/{ticker}.csv"
        
        # Save the updated configuration back to the original file
        with open(file_path, 'w') as outfile:
            yaml.dump(config, outfile, default_flow_style=False)
        
        print(f"Saved configuration with input_window={i} and output_window={i}.")
        finance_cfg = load_config('config/default/finance.yaml')

        processor = FinanceDataProcessor(finance_cfg['dataset'])
        finance_dataset = processor.get_dataset()
        processor.push_to_huggingface(finance_dataset)

Saved configuration with input_window=1 and output_window=1.
instruction_1:  Given the stock data of AAPL over the first 1 trading day, predict the stock data of AAPL over the next 1 trading day. Return the result strictly in the following JSON format, with no additional text: {
    "day_2_date": "YYYY-MM-DD",
    "day_2_stock_news": "Stock News"
}
instruction_2:  Given the stock data of AAPL over the first 1 trading day, predict the stock data of AAPL over the next 1 trading day. Return the result strictly in the following JSON format, with no additional text: {
    "day_2_date": "YYYY-MM-DD",
    "day_2_stock_news": "Stock News",
    "day_2_Close": "A Float Number"
}
instruction_3:  Given the stock data of AAPL over the first 1 trading day, predict the stock data of AAPL over the next 1 trading day. Return the result strictly in the following JSON format, with no additional text: {
    "day_2_date": "YYYY-MM-DD",
    "day_2_stock_news": "Stock News"
}
instruction_4:  Given the stock 

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [14]:

for i in range(1, 8):
    data_all = {'train': [], 'test': [], 'valid': []}
    for split in ['train', 'test', 'valid']:
        for ticker in ["AAPL", "AMD", "AMZN", "AVGO", "NFLX"]:
            data = load_dataset(f"Howard881010/finance-{ticker}-{i}day-finetuned")
            data_split = pd.DataFrame(data[split])
            
            t2t = data_split[['input_text', 'output_text', 'instruction-1']].rename(columns={'instruction-1': 'instruction', 'input_text': 'input', 'output_text': 'output'})
            tT2tT = data_split[['input_text_time', 'output_text_time', 'instruction-2']].rename(columns={'instruction-2': 'instruction', 'input_text_time': 'input', 'output_text_time': 'output'})
            tT2t = data_split[['input_text_time', 'output_text', 'instruction-3']].rename(columns={'instruction-3': 'instruction', 'input_text_time': 'input', 'output_text': 'output'})
            tT2T = data_split[['input_text_time', 'output_time', 'instruction-4']].rename(columns={'instruction-4': 'instruction', 'input_text_time': 'input', 'output_time': 'output'})
            
            # Append the transformed data for each split
            data_all[split].append(t2t)
            data_all[split].append(tT2tT)
            data_all[split].append(tT2t)
            data_all[split].append(tT2T)
            
            # Concatenate all processed DataFrames for the current split
        data_all[split] = pd.concat(data_all[split])


    # max_token = get_max_token_size(data_all, 'input', 'output', 'instruction')
    # print(max_token)
        # Convert each processed DataFrame back to Hugging Face Dataset format
    dataset_dict = DatasetDict({
        'train': Dataset.from_pandas(data_all['train']),
        'test': Dataset.from_pandas(data_all['test']),
        'valid': Dataset.from_pandas(data_all['valid'])
    })

        # Push the processed dataset to Hugging Face Hub
    dataset_dict.push_to_hub(f'Howard881010/finance-{i}day')
    data = load_dataset(f"Howard881010/finance-{i}day")
    max_token = get_max_token_size(data, 'input', 'output', 'instruction')
    print(max_token)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

README.md:   0%|          | 0.00/596 [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7716 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/976 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/964 [00:00<?, ? examples/s]

4684
